In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [1]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.7/349.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 40.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0 which is incompatible.


In [1]:
import gdown
import time
from sklearn.model_selection import train_test_split

# Define the file URLs
file_urls = [
    "https://drive.google.com/file/d/1xz_YBI8B4fHqAmxlb_3JE5ecJZS5V5z8/view?usp=sharing",
    "https://drive.google.com/file/d/1z9nYQM2wYVQ7zChtShR6ZaHBSxst_UkK/view?usp=sharing"
]

# Download files
for url in file_urls:
    file_id = url.split("/")[5]
    output_file = f"file_{file_id}.zip"  # Customize the output file name if needed
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1xz_YBI8B4fHqAmxlb_3JE5ecJZS5V5z8
To: /content/file_1xz_YBI8B4fHqAmxlb_3JE5ecJZS5V5z8.zip
100%|██████████| 271M/271M [00:01<00:00, 229MB/s]
Downloading...
From: https://drive.google.com/uc?id=1z9nYQM2wYVQ7zChtShR6ZaHBSxst_UkK
To: /content/file_1z9nYQM2wYVQ7zChtShR6ZaHBSxst_UkK.zip
100%|██████████| 18.3M/18.3M [00:00<00:00, 207MB/s]


In [2]:
import os
os.rename('/content/file_1z9nYQM2wYVQ7zChtShR6ZaHBSxst_UkK.zip', '/content/test_cleaned.csv')
os.rename('/content/file_1xz_YBI8B4fHqAmxlb_3JE5ecJZS5V5z8.zip', '/content/train_cleaned.csv')


In [3]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from hazm import *
# Load the data
train_df = pd.read_csv('train_cleaned.csv')
test_df = pd.read_csv('test_cleaned.csv')


In [4]:
#PREPROCESSING
train_df = train_df.dropna()
test_df = test_df.dropna()

train_df = train_df.replace('\[n\]', '', regex=True)
test_df = test_df.replace('\[n\]', '', regex=True)

train_df = train_df.replace('«', ' ', regex=True)
test_df = test_df.replace('«', ' ', regex=True)

train_df = train_df.replace('»', ' ', regex=True)
test_df = test_df.replace('»', ' ', regex=True)

train_df = train_df.replace('\(', ' ', regex=True)
test_df = test_df.replace('\(', ' ', regex=True)

train_df = train_df.replace('\)', ' ', regex=True)
test_df = test_df.replace('\)', ' ', regex=True)

train_df = train_df.replace(':', ' ', regex=True)
test_df = test_df.replace(':', ' ', regex=True)

train_df = train_df.replace('  ', ' ', regex=True)
test_df = test_df.replace('  ', ' ', regex=True)

train_df = train_df.replace('  ', ' ', regex=True)
test_df = test_df.replace('  ', ' ', regex=True)

In [5]:
for index, row in train_df.iterrows():
    article = row['article']
    start_index = article.find("به گزارش")
    end_index = article.find("،", start_index)
    if start_index != -1 and end_index != -1:
        article = article[:start_index] + article[end_index+1:]
    train_df.at[index, 'article'] = article
for index, row in test_df.iterrows():
    article = row['article']
    start_index = article.find("به گزارش")
    end_index = article.find("،", start_index)
    if start_index != -1 and end_index != -1:
        article = article[:start_index] + article[end_index+1:]
    test_df.at[index, 'article'] = article

In [6]:
train_df = train_df.replace(' به ', '', regex=True)
test_df = test_df.replace(' به ', '', regex=True)

train_df = train_df.replace(' با ', '', regex=True)
test_df = test_df.replace(' با ', '', regex=True)

train_df = train_df.replace(' در ', '', regex=True)
test_df = test_df.replace(' در ', '', regex=True)

train_df = train_df.replace(' برای ', '', regex=True)
test_df = test_df.replace(' برای ', '', regex=True)

train_df = train_df.replace(' از ', '', regex=True)
test_df = test_df.replace(' از ', '', regex=True)

train_df = train_df.replace(' بر ', '', regex=True)
test_df = test_df.replace(' بر ', '', regex=True)

train_df = train_df.replace(' که ', '', regex=True)
test_df = test_df.replace(' که ', '', regex=True)

train_df = train_df.replace(' را ', '', regex=True)
test_df = test_df.replace(' را ', '', regex=True)

train_df = train_df.replace(' او ', '', regex=True)
test_df = test_df.replace(' او ', '', regex=True)

train_df = train_df.replace(' آن ', '', regex=True)
test_df = test_df.replace(' آن ', '', regex=True)

train_df = train_df.replace(' آن ', '', regex=True)
test_df = test_df.replace(' آن ', '', regex=True)

In [7]:
train_df = train_df.replace('  ', ' ', regex=True)
test_df = test_df.replace('  ', ' ', regex=True)

train_df = train_df.replace('  ', ' ', regex=True)
test_df = test_df.replace('  ', ' ', regex=True)

In [8]:
normalizer = Normalizer()
stopwords = stopwords_list()

In [9]:
print(stopwords)

['آخرین', 'آقای', 'آمد', 'آمده', 'آمده_است', 'آن', 'آنان', 'آنجا', 'آنها', 'آنچه', 'آنکه', 'آورد', 'آوری', 'آیا', 'ابتدا', 'اثر', 'اجرا', 'اخیر', 'از', 'است', 'اش', 'اغلب', 'افراد', 'افرادی', 'افزود', 'البته', 'اما', 'امر', 'امکان', 'اند', 'او', 'اول', 'اولین', 'اکنون', 'اگر', 'ایشان', 'این', 'اینجا', 'اینکه', 'با', 'بار', 'باره', 'باز', 'باشد', 'باشند', 'باعث', 'بالا', 'باید', 'بخش', 'بخشی', 'بدون', 'بر', 'برابر', 'براساس', 'برای', 'برخی', 'برداری', 'بروز', 'بزرگ', 'بسیار', 'بسیاری', 'بعد', 'بعضی', 'بلکه', 'بنابراین', 'بندی', 'به', 'بهتر', 'بهترین', 'بود', 'بودن', 'بودند', 'بوده', 'بوده_است', 'بی', 'بیان', 'بیرون', 'بیش', 'بیشتر', 'بیشتری', 'بین', 'تا', 'تاکنون', 'تبدیل', 'تحت', 'ترتیب', 'تعداد', 'تعیین', 'تغییر', 'تمام', 'تمامی', 'تنها', 'تهیه', 'تو', 'جا', 'جاری', 'جای', 'جایی', 'جدی', 'جدید', 'جریان', 'جز', 'جمع', 'جمعی', 'حال', 'حالا', 'حالی', 'حتی', 'حد', 'حداقل', 'حدود', 'حل', 'خاص', 'خاطرنشان', 'خصوص', 'خطر', 'خواهد_بود', 'خواهد_شد', 'خواهد_کرد', 'خوب', 'خوبی', 'خود', 'خودش', '

In [10]:
train_df['summary'] = train_df['summary'].apply(normalizer.normalize)
train_df['article'] = train_df['article'].apply(normalizer.normalize)
test_df['summary'] = test_df['summary'].apply(normalizer.normalize)
test_df['article'] = test_df['article'].apply(normalizer.normalize)

In [11]:
train_df['summary'] = train_df['summary'].apply(lambda x: ' '.join(word for word in x.split() if word not in stopwords))
train_df['article'] = train_df['article'].apply(lambda x: ' '.join(word for word in x.split() if word not in stopwords))
test_df['summary'] = test_df['summary'].apply(lambda x: ' '.join(word for word in x.split() if word not in stopwords))
test_df['article'] = test_df['article'].apply(lambda x: ' '.join(word for word in x.split() if word not in stopwords))

In [12]:
print(train_df)

                                                 article  \
0      علی کاردر امروز ۲۷ دی ماهمراسم تودیع محسن قمصر...   
1      علی‌اصغر گودرزی‌فراهانیاشارهاینکه طرح‌هایحال ا...   
2      جمعیکارشناسان پالایشگاههدف معرفی محصول کود گوگ...   
3      سعید نظریصفحه اینستاگرام نوشتحالیکه شهر شیراز ...   
4      سیدباقر مرتضوی، مشاور وزیر نفت مدیرکل اچ اس یی...   
...                                                  ...   
82017  تیم‌های ملی هاکی زنان مردان ایرانسومین دیدار خ...   
82018  مصطفی قلی‌خسروی افزودکشورهای رقم درصداست کهفرو...   
82019  نقلپایگاه اطلاع‌رسانی کمیته امداد، سیدمرتضی بخ...   
82020  طرحامروز پنجشنبه بیستم ساعت ۱۴ روز یکشنبه ۲۳ ش...   
82021  مجموع کاهش تولید موثر کشورهای عضو غیرعضو اوپک ...   

                                                 summary  
0      مدیرعامل شرکت ملی نفت، عملکرد مدیریت امور بین‎...  
1      سرپرست مدیریت برنامه‌ریزی توسعه شرکت ملی صنایع...  
2      پالایشگاه گاز خانگیرانهدف معرفی گوگرد بنتونیتی...  
3      سخنگوی شورای شهر شیراز عمرانی شهرساز

In [ ]:
tokenized_articles = train_df['article'].apply(lambda x: x.split())
tokenized_summaries = train_df['summary'].apply(lambda x: x.split())

from hazm import Lemmatizer
lemmatizer = Lemmatizer()
lemmatized_articles = tokenized_articles.apply(lambda x: [lemmatizer.lemmatize(token) for token in x])
lemmatized_summaries = tokenized_summaries.apply(lambda x: [lemmatizer.lemmatize(token) for token in x])
sentences = lemmatized_articles + lemmatized_summaries
word2vec_model = Word2Vec(sentences, vector_size=200, window=5, min_count=50, workers=4)


In [20]:
del sentences

word_vectors = word2vec_model.wv
vocab_size = len(word_vectors.key_to_index)
embedding_dim = word2vec_model.vector_size

input_sequences = [[word_vectors.key_to_index[word] for word in article if word in word_vectors.key_to_index] for article in lemmatized_articles]
target_sequences = [[word_vectors.key_to_index[word] for word in summary if word in word_vectors.key_to_index] for summary in lemmatized_summaries]

In [21]:
# word2vec_model TEST:
vocab_size = len(word2vec_model.wv.key_to_index)
print("Vocabulary size:", vocab_size)
embedding_dim = word2vec_model.vector_size
print("Embedding dimension:", embedding_dim)
word = "اکبر"
word_vector = word2vec_model.wv[word]
print("Vector representation of", word, ":\n", word_vector)

similar_words = word2vec_model.wv.most_similar(word)
print("Similar words to", word, ":\n", similar_words)

similar_words = word2vec_model.wv.most_similar('پردازنده')
print("Similar words to پردازنده :\n", similar_words)
#END OF word2vec_model TEST:

Vocabulary size: 18173
Embedding dimension: 200
Vector representation of اکبر :
 [ 1.2350199   0.280907   -0.40399185 -0.20215182 -0.5943895   0.45831972
 -1.0339925   0.55347234 -0.44394156  1.6679451  -0.49446937  0.07471522
 -0.12149055  0.4831254   0.06544478  1.3032215  -1.3123544  -0.7039888
  0.44597727 -0.7884696   0.05496774 -0.5730534  -0.82563394  0.0495027
 -0.68077105  0.19453597  0.79829437 -0.673367   -1.4534003  -0.03857362
  1.262686    0.54629976 -0.66839254  0.20944403  0.84818304 -0.15494092
  0.6310205  -0.18958978  0.24378341  0.66389865 -0.87286013 -1.1098772
  0.12551327 -0.12275736  0.33487585 -0.14404838  0.84873587 -0.14195542
 -0.07694794 -0.08701669  1.0913901   1.0149726  -0.0270554  -0.13456213
  0.6566119   0.2969537   0.1117627   0.03661583 -0.5945651   0.17060229
 -0.8888519   0.41917536 -0.18336241  1.2918079  -0.27415344  0.14106542
 -0.58855456  1.2981962  -0.19965047  0.63436294  0.16668017 -1.254201
  0.09732016 -0.6503214  -0.12611039  0.84011227

In [22]:
article_lengths = lemmatized_articles.apply(lambda x: len(x))
summary_lengths = lemmatized_summaries.apply(lambda x: len(x))

print("Article length statistics:")
print(article_lengths.describe())

print("\nSummary length statistics:")
print(summary_lengths.describe())

Article length statistics:
count    82013.000000
mean       176.135027
std        122.827211
min          2.000000
25%        102.000000
50%        140.000000
75%        205.000000
max       1837.000000
Name: article, dtype: float64

Summary length statistics:
count    82013.000000
mean        17.949313
std          5.964220
min          4.000000
25%         14.000000
50%         17.000000
75%         21.000000
max         89.000000
Name: summary, dtype: float64


In [23]:
max_length_input = 250
max_length_output = 25
padded_inputs = pad_sequences(input_sequences, maxlen=max_length_input, padding='post')
padded_targets = pad_sequences(target_sequences, maxlen=max_length_output, padding='post')

In [24]:
validation_size = 0.15
validation_df, test_df = train_test_split(test_df, test_size=1-validation_size, random_state=42)
print(f"Validation set size: {len(validation_df)}")
print(f"Test set size: {len(test_df)}")

Validation set size: 838
Test set size: 4755


In [25]:
#test Data:
tokenized_articles_test = test_df['article'].apply(lambda x: x.split())
tokenized_summaries_test = test_df['summary'].apply(lambda x: x.split())

lemmatized_articles_test = tokenized_articles_test.apply(lambda x: [lemmatizer.lemmatize(token) for token in x])
lemmatized_summaries_test = tokenized_summaries_test.apply(lambda x: [lemmatizer.lemmatize(token) for token in x])

input_sequences_test = [[word_vectors.key_to_index[word] for word in article if word in word_vectors.key_to_index] for article in lemmatized_articles_test]
target_sequences_test = [[word_vectors.key_to_index[word] for word in summary if word in word_vectors.key_to_index] for summary in lemmatized_summaries_test]

padded_inputs_test = pad_sequences(input_sequences_test, maxlen=max_length_input, padding='post')
padded_targets_test = pad_sequences(target_sequences_test, maxlen=max_length_output, padding='post')

#Valid Data:
tokenized_articles_valid = validation_df['article'].apply(lambda x: x.split())
tokenized_summaries_valid = validation_df['summary'].apply(lambda x: x.split())

lemmatized_articles_valid = tokenized_articles_valid.apply(lambda x: [lemmatizer.lemmatize(token) for token in x])
lemmatized_summaries_valid = tokenized_summaries_valid.apply(lambda x: [lemmatizer.lemmatize(token) for token in x])

input_sequences_valid = [[word_vectors.key_to_index[word] for word in article if word in word_vectors.key_to_index] for article in lemmatized_articles_valid]
target_sequences_valid = [[word_vectors.key_to_index[word] for word in summary if word in word_vectors.key_to_index] for summary in lemmatized_summaries_valid]

padded_inputs_valid = pad_sequences(input_sequences_valid, maxlen=max_length_input, padding='post')
padded_targets_valid = pad_sequences(target_sequences_valid, maxlen=max_length_output, padding='post')

In [27]:
word2vec_model.save("/content/drive/MyDrive/FinalFilesForPD/word2vec_model.bin")

import pickle
variables = {'input1': padded_inputs, 'output1': padded_targets, 'input2': padded_inputs_test,'output2': padded_targets_test, 'input3': padded_inputs_valid, 'output3': padded_targets_valid , 'info':[250,25,82013,4755,838]}

In [28]:
with open('/content/drive/MyDrive/FinalFilesForPD/allInputs.pickle', 'wb') as f:
    pickle.dump(variables, f)